In [ ]:
from IPython.display import Latex

import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd

import math

**Decadimento del Sodio**

Si presenta un'acquisizione dati di una sorgente radioattiva di $^{22}$ Na, osservata posizionando la sorgente in una schermatura di piombo (per eliminare il fondo ambientale dall'acquisizione) e servendosi di un rivelatore a cristalli di NaI(Tl) - cristalli di ioduro di sodio drogati al tallio.

Lo spettro di emissione osservato è quello del $^{22}$ Ne, formatosi in uno stato eccitato da un decadimento $\beta^{+}$ del sodio e che, dunque, rilassa allo stato fondamentale con l'emissione di un fotone. Quest'ultimo è ciò che il rivelatore osserva: il picco relativo a tale fotone si trova alla posizione di energia E = 0.661 MeV. 

Poichè il fotone emesso può interagire con la materia (e dunque con il rivelatore) in molti modi diversi, si osserverà nello spettro qualcosa di diverso da un singolo picco, "allargato" dagli effetti di risoluzione dello strumento. 




**Composizione dello spettro osservato**

Si elencano gli effetti osservati:

-  Effetto fotoelettrico: un fotone che interagisca nel materiale con effetto fotoelettrico viene completamente assorbito, quindi il rivelatore segnalerà un conteggio nel picco ad energia corrisponente all'energia del fotone incidente;
- Effetto Compton: un fotone che interagisca nel materiale tramite scattering Compton cede energia a un elettrone del materiale dipendente dall'angolo dio scattering (in generale saranno coperti pressoché uniformemente tutti gli angoli da $0 \degree$ a $180 \degree$). Quest'ultima è l'energia "letta" dal rivelatore, data dalla differenza tra l'energia del fotone incidente e quella del fotono dopo l'interazione. L'energia del fotone dopo lo scattering Compton è data da $$E' = \frac{E}{1 + [1 - cos(\theta)\frac{E}{m_e c^2}]};$$
- Effetto fotoelettrico nella schermatura in piombo: l'effetto è lo stesso descritto all'inizio nel materiale, i fotoni emessi dal rilassamento hanno energia $E = 0.074 MeV$. Gli eventi osservati per questo effetto saranno denotati come raggi X del piombo;
- Effetto Compton nella schermatura in piombo: l'effetto è lo stesso descritto nel secondo punto. Si faccia attenzione al fatto che, però, il fotone rivelato è quello successivo all'interazione, descritto semplicemente dalla formula precedente. Tipicamente gli angoli per i quali l'effetto viene osservato è compreso tra $120 \degree$ e $180 \degree$. Gli eventi osservati per questo effetto comporrano il picco di backscatter.


**Risoluzione dello strumento**

Bisogna, a questo punto, tener conto degli effetti di risoluzione dello strumento. I valori veri osservati sono soggetti a fluttuazioni statistiche che possono essere rappresentate da un effetto di smearing gaussiano applicato ai valori osservati (o generati randomicamente). Nello specifico, ai valori generati verrà aggiunto un valore casuale preso da una distribuzione gaussiana individuata da 
- $\mu = 0$
- $\sigma = \frac{FWHM}{2.355}$

La full width at half maximum (FWHM) è determinata a partire dalla sua relazione con la risoluzione dello strumento: $$R = \frac{FWHM}{E}. $$
Per la risoluzione dello strumento si utilizzerà, in un primo momento, un valore di risoluzione **R = 0.1**.


**Acquisizione dello spettro**

In questa parte si generano randomicamente valori dello spettro tenendo conto dei principali effetti osservati. Si fissano, prima di tutto, i valori di energia noti. Si passa, poi, a riempire una lista con eventi generati randomicamente.


In [ ]:
E = 0.662           # MeV       gamma energy (from Na photoelectric effect)
E_xrays = 0.074     # MeV       X-Rays emission from lead screen

R = 0.1             # device resolution (is used to evaluate the gaussian smearing effects)

E_true = []
E_meas = []

n = 500000          # number of cicles

**Picco fotoelettrico**

In [ ]:
for i in range(n):
    E_true.append(E)
    E_meas.append(E + random.gauss(0, E*R/2.355))

**Effetto Compton**

In [ ]:
def compton(E, theta):
    """
    Thi function evaluates the energy of a photon involved in a Compton scattering interaction

    E: energy of the incident photon [MeV]
    theta: scattering angle [degrees]
    """
    return E/(1 + (1-np.cos(math.radians(theta)))*(E/0.511) )


for i in range(2*n):
    compton_edge = E - compton(E, 180)
    E_compton = compton_edge * random.uniform(0,1)

    E_true.append(E_compton)
    E_meas.append(E_compton + random.gauss(0, E_compton*R/2.355))

**Raggi X del piombo**

In [ ]:
for i in range(int(n/10)):
    E_true.append(E_xrays)
    E_meas.append(E_xrays + random.gauss(0, E_xrays*R/2.355))

**Picco di backscatter**

In [ ]:
for i in range(int(n/10)):
    theta = 120 + 60 * random.uniform(0,1)
    E_back = compton(E, theta)

    E_true.append(E_back)
    E_meas.append(E_back + random.gauss(0, E_back*R/2.355))

len(E_meas)

A questo punto, collezionati tutti i dati, è possibile visualizzarli in un semplice istogramma.
Vengono visualizzati due istogrammi sovrapposti:
- meas: istogramma che ricostruisce i dati osservati sperimentalmente, affetti da errori legati alla risoluzione dello strumento
- true: valori attesi dalle considerazioni teoriche a monte

In [ ]:
bins = np.linspace(math.floor(min(E_meas)), math.ceil(max(E_meas)), 1200)

fig1, ax1 = plt.subplots()

ax1.set_xlim([min(E_meas)-0.01, max(E_meas)+0.01])
ax1.set_ylim(top=10000)

ax1.hist(E_meas, bins=bins, alpha=0.5, label='meas')
ax1.hist(E_true, bins=bins, alpha=0.5, label='true')
plt.legend(loc='upper right')
ax1.set_xlabel('E [MeV]')
ax1.set_ylabel('counts')

plt.show()

In [ ]:
%matplotlib ipympl

live1 = E_meas
live2 = E_true

random.shuffle(live1)
random.shuffle(live2)

def update(curr):
    ax.clear()
    ax.hist(live1[:curr], bins=bins)
    ax.set_title('n={}'.format(curr))

fig, ax = plt.subplots()
ax.set_xlim([min(E_meas)-0.01, max(E_meas)+0.01])
ax.set_title('Na spectrum')
ax.set_xlabel('E [MeV]')
ax.set_ylabel('counts')

a = animation.FuncAnimation(fig, update, frames=np.arange(0, len(live1), 5000), interval=40, repeat=False, blit=False)

plt.show()

A questo punto, prendendo i dati a disposizione, si può provare a rappresentare graficamente lo spettro effettivamente osservato in laboratorio.


In [ ]:
data_path = "../data_ascii/spectrum_Na22.txt"
data = pd.read_csv(data_path)

data.columns
data.Data.sum()


In [ ]:
bins = np.linspace(math.floor(min(data.Channel)), math.ceil(max(data.Channel)), 1200)

fig2, ax2 = plt.subplots()

ax2.set_xlim([min(data.Channel)-1, max(data.Channel)+1])
ax2.set_ylim(top=1000)

ax2.hist(data.Channel, weights=data.Data, bins=bins, alpha=0.5, label='exp')
ax2.set_xlabel('channel [CHN]')
ax2.set_ylabel('counts')

plt.show()

È evidente che, tuttavia, l'istogramma creato a partire dai dati sperimentali, direttamente caricati dall'analizzatore multicanale, categorizzi i dati in un'unità particolare (canali), anziché nelle consuete energie, già adoperate per gli istogrammi precedenti. Questo può essere risolto tramite una conversione dei valori in canali in valori in energia. Si osservi che questo è stato fatto in laboratorio tramite la calibrazione dell'analizzatore multicanale. La relazione tra valore in canali C ed energie E trovata è $$E = \alpha + \beta C + \gamma C^2,$$ con 
- $\alpha = (−1.51 \pm 0.15)\cdot 10^{-2} MeV$
- $\beta = (16.60 \pm 0.08)\cdot 10^{-4} MeV \cdot CHN^{-1}$
- $\gamma = (4.1 \pm 0.8)\cdot 10^{-8} MeV \cdot CHN^{-2}$

Si convertano i valori in canali segnati dal dispositivo in energie tramite la relazione proposta.

In [ ]:
alpha = -1.51e-2
beta = 16.6e-4
gamma = 4.1e-8

a = []
for i in range(len(data.Channel)):
    a.append(alpha)
E_data = a + beta * data.Channel + gamma * data.Channel * data.Channel

E_data.head()

In [ ]:
bins = np.linspace(math.floor(min(E_data)), math.ceil(max(E_data)), 1200)

fig3, ax3 = plt.subplots()

ax3.set_xlim([min(E_data)-0.01, max(E_data)+0.01])
ax3.set_ylim(top=1500)

ax3.hist(E_data, weights=data.Data, bins=bins, alpha=0.5, label='exp')
ax3.legend(loc='upper right')
ax3.set_xlabel('E [MeV]')
ax3.set_ylabel('counts')

plt.show()